In [1]:
# import relevant modules
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sn
import scipy.stats as stats
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
%matplotlib inline

In [3]:
x_y_train= pd.read_csv("training_twitter_x_y_train.csv",delimiter=',')
x_test= pd.read_csv("test_twitter_x_test.csv",delimiter=',')

In [4]:
airsent = pd.read_csv('airline_sentiment_analysis.csv')

In [5]:
airsent = airsent[['airline_sentiment','text']]
airsent.head()

,airline_sentiment,text
0,positive,@VirginAmerica plus you've added commercials t...
1,negative,@VirginAmerica it's really aggressive to blast...
2,negative,@VirginAmerica and it's a really big bad thing...
3,negative,@VirginAmerica seriously would pay $30 a fligh...
4,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [6]:
airsent.shape

(11541, 2)

In [8]:
train = airsent[0:9000]
test = airsent[9001:11541]

In [9]:
train.head(2)

,airline_sentiment,text
0,positive,@VirginAmerica plus you've added commercials t...
1,negative,@VirginAmerica it's really aggressive to blast...


In [10]:
test.head(2)

,airline_sentiment,text
9001,positive,"@USAirways TYVM USAir, Happy Night@ &lt;3"
9002,positive,@USAirways thanks! It's hectic for everyone bu...


In [14]:
test.reset_index(inplace = True, drop = True)
test.head(2)

,airline_sentiment,text
0,positive,"@USAirways TYVM USAir, Happy Night@ &lt;3"
1,positive,@USAirways thanks! It's hectic for everyone bu...


In [4]:
x_y_train.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [11]:
train_data= train["text"]
target_data= train["airline_sentiment"]
print(target_data[4])
train_data[4]

positive


'@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)'

In [15]:
test_data_orig= test["text"]
test_data_orig[0]

'@USAirways TYVM USAir, Happy Night@ &lt;3'

In [16]:
from nltk.tokenize import word_tokenize

In [17]:
documents = []
for i in range(len(train_data)):
    documents.append((word_tokenize(train_data[i].lower()) ,target_data[i]))
documents[8]    

(['@',
  'virginamerica',
  'this',
  'is',
  'such',
  'a',
  'great',
  'deal',
  '!',
  'already',
  'thinking',
  'about',
  'my',
  '2nd',
  'trip',
  'to',
  '@',
  'australia',
  '&',
  'amp',
  ';',
  'i',
  'have',
  "n't",
  'even',
  'gone',
  'on',
  'my',
  '1st',
  'trip',
  'yet',
  '!',
  ';',
  'p'],
 'positive')

In [18]:
test_documents = []
for i in range(len(test_data_orig)):
    test_documents.append((word_tokenize(test_data_orig[i].lower())))
test_documents[0] 

['@',
 'usairways',
 'tyvm',
 'usair',
 ',',
 'happy',
 'night',
 '@',
 '&',
 'lt',
 ';',
 '3']

In [19]:
documents[70]

(['@', 'virginamerica', 'have', 'a', 'great', 'week', '🌞✈'], 'positive')

In [21]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [22]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [23]:
from nltk import pos_tag
w = "better"
pos_tag([w])

[('better', 'RBR')]

In [24]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)
stops, string.punctuation

({'!',
  '"',
  '#',
  '$',
  '%',
  '&',
  "'",
  '(',
  ')',
  '*',
  '+',
  ',',
  '-',
  '.',
  '/',
  ':',
  ';',
  '<',
  '=',
  '>',
  '?',
  '@',
  '[',
  '\\',
  ']',
  '^',
  '_',
  '`',
  'a',
  'about',
  'above',
  'after',
  'again',
  'against',
  'ain',
  'all',
  'am',
  'an',
  'and',
  'any',
  'are',
  'aren',
  "aren't",
  'as',
  'at',
  'be',
  'because',
  'been',
  'before',
  'being',
  'below',
  'between',
  'both',
  'but',
  'by',
  'can',
  'couldn',
  "couldn't",
  'd',
  'did',
  'didn',
  "didn't",
  'do',
  'does',
  'doesn',
  "doesn't",
  'doing',
  'don',
  "don't",
  'down',
  'during',
  'each',
  'few',
  'for',
  'from',
  'further',
  'had',
  'hadn',
  "hadn't",
  'has',
  'hasn',
  "hasn't",
  'have',
  'haven',
  "haven't",
  'having',
  'he',
  'her',
  'here',
  'hers',
  'herself',
  'him',
  'himself',
  'his',
  'how',
  'i',
  'if',
  'in',
  'into',
  'is',
  'isn',
  "isn't",
  'it',
  "it's",
  'its',
  'itself',
  'just',
  'll',


In [25]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [26]:
documents = [(clean_review(document), category) for document, category in documents]

In [27]:
documents[0]

(['virginamerica',
  'plus',
  "'ve",
  'add',
  'commercial',
  'experience',
  '...',
  'tacky'],
 'positive')

In [28]:
len(documents)

9000

In [29]:
len(test_documents)

2540

# Sample training testing

In [30]:
training_documents_orig = documents
testing_documents_orig= test_documents

training_documents = documents[0:7000]
testing_documents = documents[7000:]

In [31]:
all_words = []
for doc in training_documents:
    all_words += doc[0]

In [32]:
all_words_orig = []
for doc in training_documents_orig:
    all_words_orig += doc[0]

In [33]:
import nltk
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)
features = [i[0] for i in common]

In [34]:
import nltk
freq = nltk.FreqDist(all_words_orig)
common = freq.most_common(3000)
features_orig = [i[0] for i in common]

In [35]:
features

['united',
 'flight',
 'southwestair',
 'jetblue',
 "n't",
 'get',
 "'s",
 'thanks',
 'hour',
 'time',
 'service',
 'customer',
 'http',
 'cancel',
 'u',
 'bag',
 '...',
 'plane',
 'help',
 'wait',
 'thank',
 'make',
 'go',
 'fly',
 "'m",
 'virginamerica',
 'amp',
 'delayed',
 'airline',
 'usairways',
 'would',
 'one',
 'still',
 '2',
 'delay',
 'gate',
 'hold',
 'day',
 'seat',
 'ca',
 'try',
 'bad',
 'take',
 'say',
 'need',
 'call',
 'great',
 "'ve",
 "''",
 'guy',
 'late',
 'back',
 'like',
 '``',
 'book',
 'flightled',
 'miss',
 'work',
 'well',
 'please',
 'today',
 'give',
 'lose',
 'change',
 'know',
 'agent',
 'never',
 'airport',
 'phone',
 'minute',
 'really',
 'love',
 'problem',
 'ticket',
 'want',
 "'re",
 '3',
 'min',
 'even',
 'last',
 'good',
 'could',
 'issue',
 'see',
 'way',
 'crew',
 'look',
 'travel',
 'check',
 'people',
 'much',
 'hr',
 'first',
 'luggage',
 '4',
 'baggage',
 'experience',
 "'ll",
 'yes',
 'another',
 'home',
 'told',
 'ever',
 'let',
 'use',
 '

In [36]:
features_orig

['united',
 'flight',
 'usairways',
 'southwestair',
 'jetblue',
 "n't",
 'get',
 "'s",
 'hour',
 'thanks',
 'service',
 'time',
 'customer',
 'u',
 'cancel',
 'http',
 'help',
 'bag',
 '...',
 'wait',
 'plane',
 'hold',
 'make',
 'go',
 'amp',
 'fly',
 "'m",
 'thank',
 'delayed',
 'call',
 '2',
 'still',
 'airline',
 'gate',
 'one',
 'would',
 'day',
 'try',
 'delay',
 'bad',
 'need',
 'virginamerica',
 'ca',
 'say',
 'seat',
 'take',
 "'ve",
 'late',
 'back',
 "''",
 'flightled',
 '``',
 'guy',
 'miss',
 'please',
 'book',
 'phone',
 'work',
 'minute',
 'today',
 'like',
 'agent',
 'great',
 'give',
 'never',
 'change',
 'min',
 'lose',
 'well',
 '3',
 'know',
 'airport',
 'really',
 'problem',
 'last',
 'even',
 'ticket',
 'issue',
 'hr',
 "'re",
 'want',
 '4',
 'people',
 'way',
 'could',
 'sit',
 'luggage',
 'good',
 'check',
 'travel',
 'home',
 'love',
 'crew',
 'told',
 'experience',
 'another',
 'see',
 'ever',
 'look',
 'due',
 'let',
 'weather',
 'baggage',
 'much',
 'first'

In [37]:
def get_feature_dict(words):
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features

In [38]:
output = get_feature_dict(training_documents[0][0])
output

{'united': False,
 'flight': False,
 'southwestair': False,
 'jetblue': False,
 "n't": False,
 'get': False,
 "'s": False,
 'thanks': False,
 'hour': False,
 'time': False,
 'service': False,
 'customer': False,
 'http': False,
 'cancel': False,
 'u': False,
 'bag': False,
 '...': True,
 'plane': False,
 'help': False,
 'wait': False,
 'thank': False,
 'make': False,
 'go': False,
 'fly': False,
 "'m": False,
 'virginamerica': True,
 'amp': False,
 'delayed': False,
 'airline': False,
 'usairways': False,
 'would': False,
 'one': False,
 'still': False,
 '2': False,
 'delay': False,
 'gate': False,
 'hold': False,
 'day': False,
 'seat': False,
 'ca': False,
 'try': False,
 'bad': False,
 'take': False,
 'say': False,
 'need': False,
 'call': False,
 'great': False,
 "'ve": True,
 "''": False,
 'guy': False,
 'late': False,
 'back': False,
 'like': False,
 '``': False,
 'book': False,
 'flightled': False,
 'miss': False,
 'work': False,
 'well': False,
 'please': False,
 'today': False

In [39]:
training_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
testing_data = [(get_feature_dict(doc), category) for doc, category in testing_documents]

In [40]:
training_data_orig = [(get_feature_dict(doc), category) for doc, category in training_documents_orig]
testing_data_orig = [(get_feature_dict(doc)) for doc in testing_documents_orig]

In [31]:
#testing_data_orig[3659]

In [32]:
#training_data_orig[0][0]

In [41]:
from nltk import NaiveBayesClassifier


In [42]:
classfier = NaiveBayesClassifier.train(training_data)

In [43]:
#testing_data[0][0]

In [44]:
print(classfier.classify(testing_data[2][0]))

negative


In [45]:
print(classfier.classify(testing_data_orig[2]))

negative


In [46]:
nltk.classify.accuracy(classfier, testing_data)

0.9445

In [47]:
classfier.classify(testing_data[0][0])

'negative'

In [48]:
#classfier.show_most_informative_features(15)

In [49]:
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier

In [50]:
svc = SVC()
classifier_sklearn = SklearnClassifier(svc)

In [51]:
classifier_sklearn.train(training_data)

<SklearnClassifier(SVC())>

In [52]:
nltk.classify.accuracy(classifier_sklearn, testing_data)

0.9445

In [53]:
from sklearn.neural_network import MLPClassifier

In [54]:
clf = MLPClassifier(hidden_layer_sizes=(5,5,5,5))
classifier_sklearn11= SklearnClassifier(clf)

In [55]:
classifier_sklearn11.train(training_data)

<SklearnClassifier(MLPClassifier(hidden_layer_sizes=(5, 5, 5, 5)))>

In [56]:
nltk.classify.accuracy(classifier_sklearn11, testing_data)

0.9395

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
rfc = RandomForestClassifier()
classifier_sklearn1 = SklearnClassifier(rfc)

In [61]:
classifier_sklearn1.train(training_data)

<SklearnClassifier(RandomForestClassifier())>

In [62]:
nltk.classify.accuracy(classifier_sklearn1, testing_data)

0.9405

In [63]:
classifier_sklearn1.classify(testing_data[8][0])

'positive'

#### The Accuracy is highest in the Naives Bayes model and also it is a common model for NLP so we will use NB model in the App

In [64]:
import joblib 

In [ ]:
# save the model to disk

filename = 'nb_classifier.sav'
pickle.dump(classfier, open(filename, 'wb'))

filename = 'scv_classifier.sav'
pickle.dump(classifier_sklearn, open(filename, 'wb'))

filename = 'mlp_classifier.sav'
pickle.dump(classifier_sklearn11, open(filename, 'wb'))

filename = 'rf_classifier.sav'
pickle.dump(classifier_sklearn1, open(filename, 'wb'))

In [ ]:
# Save the model as a pickle in a file 

joblib.dump(classfier, 'nb_classifier_1.pkl') 

joblib.dump(classfier, 'scv_classifier_1.pkl') 

joblib.dump(classfier, 'mlp_classifier_1.pkl') 

joblib.dump(classfier, 'rf_classifier_1.pkl') 